Import Library & Package:

In [ ]:
!pip install sastrawi

import numpy as np
import pandas as pd
import re
import string
import nltk
import matplotlib.pyplot as plt
import os
import math

from tqdm import tqdm
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from google.colab import files

nltk.download('punkt')

#upload data.csv
print("\nUpload data.csv")
files.upload()

#upload new_kamusalay.csv
print("\nUpload new_kamusalay.csv")
files.upload()

     |████████████████████████████████| 215kB 8.6MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

Upload data.csv


Saving data.csv to data.csv

Upload new_kamusalay.csv


Saving new_kamusalay.csv to new_kamusalay.csv


{'new_kamusalay.csv': b'anakjakartaasikasik,anak jakarta asyik asyik\r\npakcikdahtua,pak cik sudah tua\r\npakcikmudalagi,pak cik muda lagi\r\nt3tapjokowi,tetap jokowi\r\n3x,tiga kali\r\naamiin,amin\r\naamiinn,amin\r\naamin,amin\r\naammiin,amin\r\nabis,habis\r\nabisin,habiskan\r\nacau,kacau\r\nachok,ahok\r\nad,ada\r\nadek,adik\r\nadl,adalah\r\nadlah,adalah\r\nadlh,adalah\r\nado,ada\r\naduhh,aduh\r\naer,air\r\nafdol,afdal\r\nagamaataualqur,agama alquran\r\nagm,agama\r\nagma,agama\r\nahaha,haha\r\nahahaha,haha\r\nahehehehe,hehe\r\nahir,akhir\r\nahirnya,akhirnya\r\nahk,ahok\r\nahlamdulillah,alhamdulillah\r\nahli2,para ahli\r\nahlusunnah,ahlus sunah\r\nahmaddani,ahmad dhani\r\naho,ahok\r\nahoax,ahok\r\nahoaxx,ahok\r\nahog,ahok\r\nahokataudjarot,ahok djarot\r\nahokbebanijokowi,ahok beban jokowi\r\nahokbtp,ahok basuki tjahaja purnama\r\nahokditolakwarga,ahok ditolak warga\r\nahokdjarot,ahok djarot\r\nahokdjarotterbukti,ahok djarot terbukti\r\nahokers,ahoker\r\nahokfighter,ahok pejuang\r\nahok

Upload & Cleaning Dataset:

In [ ]:

# baca data
df = pd.read_csv("/content/data.csv", encoding='latin-1')

# mengambil 3 kolom awal
df = df.iloc[:, :3]

## ambil stopwords
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

# menyiapkan dict untuk mengubah bahasa 'alay' menjadi baku
kamus_alay = pd.read_csv('/content/new_kamusalay.csv', encoding='latin-1')
res = {}
for i in range(len(kamus_alay)):
    res[kamus_alay.iloc[i, 0]] = kamus_alay.iloc[i, 1]

# fungsi preprocessing
def preprocessing(data):
  clean = []
  for d in data:
    # Lowercase the document
    document_test = d.lower()
    # Ganti kata alay dari kamus alay (res)
    word_tokens = word_tokenize(document_test)
    filtered_sentence = []
    for w in word_tokens:
        if w in res:
            filtered_sentence.append(res[w])
        else:
            filtered_sentence.append(w)
    document_test = (' ').join(filtered_sentence)
    # Remove Stopwords
    word_tokens = word_tokenize(document_test)
    filtered_sentence = [w for w in word_tokens if not w in stopwords]
    document_test = (' ').join(filtered_sentence)
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', document_test)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Remove the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    # hapus 1 karakter yang berulang (contoh: aaaaaa)
    document_test = re.sub("(.)\\1{2,}", "\\1", document_test)
    clean.append(document_test)
  return clean

# bersihin data
df['Tweet_clean'] = preprocessing(df['Tweet'])

df.head()

,Tweet,HS,Abusive,Tweet_clean
0,- disaat semua cowok berusaha melacak perhatia...,1,1,semua cowok berusaha melacak perhatian gue ka...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,rt pengguna pengguna siapa telat memberi tau k...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,kadang aku berpikir aku tetap percaya tuhan p...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,pengguna pengguna aku aku n nku tau matamu sip...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,pengguna pengguna kaum cebong kafir kelihatan ...


Membentuk model TF-IDF:

In [ ]:
# buat model tf-idf
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Tweet_clean'])
X = X.toarray()

tfidf = pd.DataFrame(X, columns=vectorizer.get_feature_names())

#Klasifikasi dengan K-NN
X_train = tfidf.iloc[:len(tfidf)-1, :]
y_train = df.iloc[:len(df)-1, 1:3]
k = math.sqrt(len(df))
classifier = KNeighborsClassifier(n_neighbors=8)
knn = classifier.fit(X_train, y_train)

tfidf

,aagym,aah,aamiin,ab,aba,abad,abadi,abadian,abah,abaikan,abal,aban,abang,abangan,abangku,abangnya,abaqus,abbas,abda,abdi,abdinya,abdul,abdullah,abdurrahman,abg,abis,abj,about,abraham,abri,abrik,absen,absurd,abt,abu,abud,aburizal,abut,ac,academy,...,zalimnya,zaman,zamannya,zanbi,zat,zebra,zedong,zefanya,zein,zen,zeng,zero,ziarah,zidan,zie,zikir,zilvia,zimbabwe,zina,zindiq,zinnirah,zionis,zionisme,zionist,zmbsjaiha,zof,zohri,zombi,zon,zona,zonk,zoom,zouk,zu,zuhair,zuhud,zul,zulkarnain,zulkifli,zumi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Memproses Query:

In [ ]:
#User Memasukkan Query
usrin = input("Tulis tweet anda: ")
bersih = preprocessing([usrin])

#Membuat TDM baru
Y = vectorizer.transform(bersih)
Y = Y.toarray()
tdm = pd.DataFrame(Y, columns=vectorizer.get_feature_names())

#X_test=tdm.iloc[-1,:]
y_pred = knn.predict(tdm)

if y_pred[0][0] == 0 and y_pred[0][1] == 0:
  print("Tweet bukan merupakan Ujaran kebencian dan tidak mengandung kata kasar")
elif y_pred[0][0] == 1 and y_pred[0][1] == 0:
  print("Tweet merupakan Ujaran kebencian")
elif y_pred[0][0] == 0 and y_pred[0][1] == 1:
  print("Tweet mengandung kata kasar")
else:
  print("Tweet merupakan Ujaran kebencian dan mengandung kata kasar")

Evaluasi:

In [ ]:
## split data menjadi data test dan train untuk bisa menilai kebaikan dari model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf, df.iloc[:, 1:3], test_size=0.2, random_state=21)

# memprediksi dari data test
y_pred = knn.predict(X_test)

from sklearn.metrics import recall_score, precision_score, f1_score

print("recall score :", recall_score(y_test, y_pred, average='macro'))
print("precision score :", precision_score(y_test, y_pred, average='macro'))
print("F1 score :",f1_score(y_test, y_pred, average='macro'))

recall score : 0.7687384021919639
precision score : 0.8420808278942196
F1 score : 0.8037395693584144
